In [1]:
## Import packages
import os
import csv
import numpy as np
from scipy.io import wavfile

import tensorflow as tf
from tensorflow import gfile

import vggish_input
import vggish_postprocess
import vggish_params
import vggish_slim

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')

flags.DEFINE_string(
    'audio_embedding_feature_name', 'audio_embedding',
    'Path to the VGGish checkpoint file.')

## Map met csv-files
flags.DEFINE_string(
    'csvfile_path', 'csv_files/',
    'Path to a csv files.')

## Vul hier de map in waarin alle wav-files staan
flags.DEFINE_string(
    'wavfile_path', 'wav_files/',
    'Path to wav files')

## Map waar de nieuwe tf-records terecht komen
flags.DEFINE_string(
    'tfrecord_path', 'tfrecords/added_data/',
    'Path to a TFRecord file where embeddings will be written.')

## De volgende bestanden komen uit het eerdere mapje 'models'. Nu heb ik ze in yt8m gezet.
flags.DEFINE_string(
    'pca_params', 'yt8m/vggish_pca_params.npz',
    'Path to the VGGish PCA parameters file.')

flags.DEFINE_string(
    'checkpoint', 'yt8m/vggish_model.ckpt',
    'Path to the VGGish checkpoint file.')

flags.DEFINE_string(
    'yt_checkpoint', 'yt8m/youtube_model.ckpt',
    'Path to the VGGish checkpoint file.')

In [3]:
## Voor alle wav-files in een map, gebeurt het volgende:

## Eerst wordt het wav-file ingelezen en de bijbehorende label(s) opgezocht in een csv-bestand
## Als video-id wordt een willekeurig id'tje gegeven (in dit geval allemaal dezelfde)

## Dan parsen we wav-file naar embeddings: 
# (dit gebeurt door vggish_input.wavfile_to_example aan te roepen)
# Stap 1a: lezen van wav-file, input is array met samples die db aanduiden. Ook sample rate (per sec) wordt gelezen
# Stap 1b: Bij 2d array (stereo, ipv mono) bereken gemiddelde, daarna normaliseren (delen door 32.768)
# Stap 2: Bepaal examples in vorm [batch size, num frames, num bands].
    # Hierbij worden voor verschillende batches (omdat alles tegelijk niet in 1x in NN kan),
    # een log mel spectrogram gemaakt (in vorm [num_frames, num_bands])
# Stap 3: Bepaal features: nu wordt de embedding laag gemaakt (PCA-components, discreet maken etc)
    # Hiervoor worden model-parameters opgehaald die eerder zijn opgeslagen
    
## Daarna wordt een sequence example gemaakt (in getSequenceExample) en het als tf-records weggeschreven

In [4]:
## Function that takes examples from wav-file as input and returns a sequence example

def getSequenceExample(examples_batch, labels, video_id=[b'-1LrH01Ei1w']):
    with tf.Graph().as_default(), tf.Session() as sess:

        # Define the model: load the checkpoint and locate input and output tensors
        # Input: [batch_size, num_frames, num_bands] 
        # where [num_frames, num_bands] represents log-mel-scale spectrogram
        # Output: embeddings
        vggish_slim.define_vggish_slim(training=False)
        vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)

        pca_params = np.load(FLAGS.pca_params)
        pca_matrix = pca_params[vggish_params.PCA_EIGEN_VECTORS_NAME]
        pca_means = pca_params[vggish_params.PCA_MEANS_NAME].reshape(-1, 1)

        features_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_INPUT_TENSOR_NAME)
        embedding_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_OUTPUT_TENSOR_NAME)
        vggish_slim.load_youtube_model(sess, FLAGS.yt_checkpoint)

        # Run inference and postprocessing
        [embedding_batch] = sess.run([embedding_tensor],
                                     feed_dict={features_tensor: examples_batch})

        postprocessed_batch = np.dot(
                pca_matrix, (embedding_batch.T - pca_means)
            ).T
        #print(postprocessed_batch)

        num_frames = np.minimum(postprocessed_batch.shape[0], vggish_params.MAX_FRAMES)
        data = vggish_postprocess.resize(postprocessed_batch, 0, vggish_params.MAX_FRAMES)
        data = np.expand_dims(data, 0)
        num_frames = np.expand_dims(num_frames, 0)

        input_tensor = sess.graph.get_collection("input_batch_raw")[0]
        num_frames_tensor = sess.graph.get_collection("num_frames")[0]

        label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=labels))
        videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id))

        seq_example = tf.train.SequenceExample(
            context = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat}),
            feature_lists = tf.train.FeatureLists(
                feature_list={
                    FLAGS.audio_embedding_feature_name:
                        tf.train.FeatureList(
                            feature=[
                                tf.train.Feature(
                                    bytes_list=tf.train.BytesList(
                                        value=[embedding.tobytes()]))
                                for embedding in postprocessed_batch
                            ]
                        )
                }
            )
        )
    return(seq_example)

In [5]:
def getLabels(file):
    return()

In [6]:
## Lees stuk voor stuk alle wav-files in
## Zoek het bijbehorende label op in een csv-bestand
## Bepaal de embeddings

# Prepare a record writer to store the postprocessed embeddings.
tfrecord_file = str(FLAGS.tfrecord_path + 'gun.tfrecord')
writer = tf.python_io.TFRecordWriter(tfrecord_file)

## Read and rewrite all test files
files = gfile.Glob(str(FLAGS.wavfile_path + "*.wav"))

for file in files:
    
    print("Filename: " + str(file))
   
    ## Find labels belonging to wav-file
    ## 15=Gunshots/gunfire, 35=Gunshot Sensing clues
    labels = [15,35]
    
    ## This function reads the wav file and converts the samples into np arrays of [batch size, num frames, num bands]
    #examples_batch = vggish_input.wavfile_to_examples(str(FLAGS.wavfile_path + wav_file))
    examples_batch = vggish_input.wavfile_to_examples(file)
    print("Examples shape: " + str(examples_batch.shape))
   
    seq_example = getSequenceExample(examples_batch, labels)
    
    writer.write(seq_example.SerializeToString())

writer.close()

Filename: wav_files\car_18.wav
Examples shape: (11, 96, 64)
INFO:tensorflow:Restoring parameters from yt8m/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from yt8m/youtube_model.ckpt
